# AZR Model Trainer v2 — Google Colab

**Neural network trainer with web UI, GPU acceleration, dataset catalog (136 datasets)**

---

### How to use:
1. **Runtime → Change runtime type → GPU (T4)**
2. Run all cells in order
3. Click the **ngrok link** at the end to open the web interface

---

## 1. Check GPU

In [ ]:
import torch
print(f'PyTorch: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'VRAM: {torch.cuda.get_device_properties(0).total_mem / 1024**3:.1f} GB')
else:
    print('WARNING: GPU not found! Go to Runtime -> Change runtime type -> GPU')

## 2. Clone repository & install dependencies

In [ ]:
import os

# Clone repo
if not os.path.exists('/content/ai-neural-network-project'):
    !git clone https://github.com/Slavikpro557/ai-neural-network-project.git
else:
    print('Repo already cloned, pulling latest...')
    !cd /content/ai-neural-network-project && git pull

# Install dependencies (torch is already installed on Colab)
!pip install -q fastapi uvicorn python-multipart pydantic numpy psutil PyMuPDF pyngrok

print('\nDone!')

## 3. Setup ngrok tunnel

ngrok creates a public URL to access the server running on Colab.

### How to set up (one time):
1. Get a free token at https://ngrok.com (sign up → Your Authtoken)
2. In Colab: click **🔑 Secrets** (key icon in left panel)
3. Add a secret: name = `NGROK_TOKEN`, value = your token
4. Toggle the switch to enable access for this notebook

Your token is stored **securely in your Google account**, never in the code!

In [ ]:
from pyngrok import ngrok
import getpass

NGROK_TOKEN = None

# Method 1: Colab Secrets (recommended)
try:
    from google.colab import userdata
    NGROK_TOKEN = userdata.get('NGROK_TOKEN')
    print('Token loaded from Colab Secrets!')
except Exception:
    pass

# Method 2: manual input
if not NGROK_TOKEN:
    print('Token not found in Secrets.')
    print('Paste your ngrok token below (hidden input):')
    NGROK_TOKEN = getpass.getpass('ngrok token: ')

if NGROK_TOKEN:
    ngrok.set_auth_token(NGROK_TOKEN)
    print('ngrok ready!')
else:
    print('WARNING: No token provided. Tunnel may not work.')

## 4. Start AZR Trainer server

In [ ]:
import subprocess
import time
from pyngrok import ngrok

os.chdir('/content/ai-neural-network-project')

# Kill any existing server
!kill $(lsof -t -i:8000) 2>/dev/null; true
time.sleep(1)

# Start server in background
server_process = subprocess.Popen(
    ['python', 'server_with_datasets.py'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    cwd='/content/ai-neural-network-project'
)

# Wait for server to start
print('Starting server...')
time.sleep(5)

# Check if server started
if server_process.poll() is None:
    print('Server is running!')
else:
    print('ERROR: Server failed to start. Check logs below:')
    print(server_process.stdout.read().decode('utf-8', errors='replace'))

# Open ngrok tunnel
try:
    # Close existing tunnels
    tunnels = ngrok.get_tunnels()
    for t in tunnels:
        ngrok.disconnect(t.public_url)
except:
    pass

public_url = ngrok.connect(8000)

print()
print('=' * 60)
print(f'  AZR Model Trainer is running!')
print(f'  GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"}')
print(f'')
print(f'  Open this link:')
print(f'  {public_url}')
print(f'')
print(f'  (keep this cell running!)')
print('=' * 60)

## 5. Server logs (optional)
Run this cell to see server output if something goes wrong:

In [ ]:
# Show last server logs
import subprocess
result = subprocess.run(['tail', '-30', '/proc/' + str(server_process.pid) + '/fd/1'],
                       capture_output=True, text=True)
if result.returncode != 0:
    # Alternative: read from process
    print('Server PID:', server_process.pid)
    print('Status:', 'Running' if server_process.poll() is None else 'Stopped')
else:
    print(result.stdout)

## 6. Stop server (when done)

In [ ]:
# Run this to stop everything
ngrok.kill()
server_process.terminate()
print('Server stopped. Tunnel closed.')